<a href="https://colab.research.google.com/github/mmsamiei/just-practice-deep/blob/master/Bradley_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torchtext
from torchtext.data import Field
import spacy

from spacy.symbols import ORTH
my_tok = spacy.load('en')

def spacy_tok(x):
    return [tok.text for tok in my_tok.tokenizer(x)]

QUERY = Field(lower=True, tokenize=spacy_tok)
RESPONSE = Field(lower=True, tokenize=spacy_tok, is_target=True, init_token='<bos>', eos_token='<eos>')

In [0]:
import torch
if torch.cuda.is_available:
  device = torch.device('cuda')
else:
  device = torch.device('cpu')

In [0]:
my_tok.tokenizer.add_special_case("don't", [{ORTH: "do"}, {ORTH: "n't"}])

In [0]:
from torchtext.data import TabularDataset

train_dataset = TabularDataset(path="./formatted_movie_lines.txt", format="CSV",
                               fields=[("query", QUERY),("response", RESPONSE)],
                               csv_reader_params={"delimiter":'\t'})

In [5]:
QUERY.build_vocab(train_dataset)
RESPONSE.build_vocab(train_dataset)
print(QUERY.vocab.stoi['film'])
print(QUERY.vocab.itos[33])
print(len(QUERY.vocab))
print(len(RESPONSE.vocab))

1059
're
48505
49036


In [0]:
from torchtext.data import BucketIterator

batch_size = 16

train_iterator = BucketIterator(dataset= train_dataset, batch_size=batch_size
                                ,device=device
                                ,sort_key=lambda x: data.interleave_keys(len(x.query), len(x.response))
                                , repeat = False)

In [7]:
for batch in (iter(train_iterator)):
  print(batch.query.shape)

torch.Size([56, 16])
torch.Size([24, 16])
torch.Size([32, 16])
torch.Size([47, 16])
torch.Size([134, 16])
torch.Size([45, 16])
torch.Size([66, 16])
torch.Size([39, 16])
torch.Size([49, 16])
torch.Size([105, 16])
torch.Size([91, 16])
torch.Size([32, 16])
torch.Size([65, 16])
torch.Size([65, 16])
torch.Size([49, 16])
torch.Size([61, 16])
torch.Size([29, 16])
torch.Size([40, 16])
torch.Size([45, 16])
torch.Size([72, 16])
torch.Size([53, 16])
torch.Size([46, 16])
torch.Size([96, 16])
torch.Size([72, 16])
torch.Size([48, 16])
torch.Size([33, 16])
torch.Size([18, 16])
torch.Size([67, 16])
torch.Size([34, 16])
torch.Size([32, 16])
torch.Size([42, 16])
torch.Size([31, 16])
torch.Size([25, 16])
torch.Size([76, 16])
torch.Size([38, 16])
torch.Size([49, 16])
torch.Size([22, 16])
torch.Size([22, 16])
torch.Size([34, 16])
torch.Size([199, 16])
torch.Size([53, 16])
torch.Size([26, 16])
torch.Size([23, 16])
torch.Size([36, 16])
torch.Size([47, 16])
torch.Size([45, 16])
torch.Size([27, 16])
torch.Size

In [0]:
import torch.nn as nn
class Bradley(nn.Module):
  def __init__(self, src_voc_sze, trg_voc_sze, hid_sze, num_head, num_enc, num_dec):
    super(Bradley, self).__init__()
    self.hid_sze = hid_sze
    self.src_word_embedding = nn.Embedding(src_voc_sze, self.hid_sze)
    self.trg_word_embedding = nn.Embedding(trg_voc_sze, self.hid_sze)
    self.trg_pos_embedding = nn.Embedding(1000, self.hid_sze)
    self.num_head = num_head
    self.transformer = nn.Transformer(self.hid_sze, self.num_head, num_enc, num_dec)
    self.fc = nn.Linear(self.hid_sze, trg_voc_sze)
  
  def forward(self, src, trg):
    #src = [src sent len, batch_size]
    #trg = [trg sent len, batch_size]
    temp_src = self.src_word_embedding(src)
    temp_trg = self.trg_word_embedding(trg)
    trg_sent_len, batch_size = trg.shape[0], trg.shape[1]
    trg_pos = self.trg_pos_embedding(torch.arange(0, trg_sent_len).unsqueeze(0).
                                     repeat(batch_size,1).to(device)).transpose(0,1)
    temp_trg = temp_trg + trg_pos
    temp = self.transformer(temp_src, temp_trg)
    return self.fc(temp)
  
  def greedy_inference_one_sample(self, src, max_len=10):

    self.eval()
    #src = [src sent len]
    src_len = src.shape[0]
    src = src.unsqueeze(1)
    #src = [sent_len, 1]
    trg = src.new_full((1,1), RESPONSE.vocab.stoi['<bos>'])
    #trg = [1,1]

    translation_step = 0
    while translation_step < max_len:
      out = self.forward(src, trg)
      out = out[-1,:]
      for i, val in enumerate(out[0]):
        if(val > 7):
          print(i, val)
      #out = [batch_size, trg_vocab_size]
      nex = out.argmax(dim=1).unsqueeze(0)
      #nex = [1, 1]
      trg = torch.cat((trg,nex), dim=0)
      translation_step += 1
    return trg



In [0]:
bradley_model = Bradley(src_voc_sze=len(QUERY.vocab), trg_voc_sze=len(RESPONSE.vocab),
                        hid_sze=64, num_head=4,
                        num_enc=3, num_dec=2)
bradley_model = bradley_model.to(device)

In [10]:
from tqdm import tqdm

criterion = nn.CrossEntropyLoss(ignore_index=RESPONSE.vocab.stoi['<pad>'])
lr = 0.5
optimizer = torch.optim.SGD(bradley_model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.95)
epoch_number = 2

print(len(train_dataset)/batch_size)

for epoch in range(1, epoch_number+1):
  print("epoch ", epoch)
  bradley_model.train()
  for i, batch in enumerate(iter(train_iterator)):
    src = batch.query
    trg = batch.response
    optimizer.zero_grad()
    out = bradley_model(src, trg)
    loss = criterion(out[:-1,:].view(-1, out.shape[2]), trg[1:,:].view(-1))
    if(i%50==0):
      print("{}: {}".format(i, loss.item()))
    loss.backward()
    torch.nn.utils.clip_grad_norm_(bradley_model.parameters(), 0.5)
    optimizer.step()

13830.125
epoch  1
0: 10.966368675231934
50: 7.075013160705566
100: 6.493185043334961
150: 6.079606056213379
200: 6.031318187713623
250: 6.415282249450684
300: 6.597004413604736
350: 6.131542682647705
400: 5.691965103149414
450: 5.852413654327393
500: 5.926379680633545
550: 5.572832107543945
600: 5.309994220733643
650: 5.70719051361084
700: 5.317142963409424
750: 5.021653175354004
800: 5.430238723754883
850: 4.490469455718994
900: 4.538619518280029
950: 3.8323919773101807
1000: 4.094155788421631
1050: 4.69091796875
1100: 2.930903434753418
1150: 3.2181129455566406
1200: 3.4726953506469727
1250: 3.8944671154022217
1300: 2.9044461250305176
1350: 2.5926456451416016
1400: 3.9137117862701416
1450: 4.040011405944824
1500: 2.31860613822937
1550: 2.8568036556243896
1600: 2.224835157394409
1650: 1.9690818786621094
1700: 2.40466046333313
1750: 3.8578920364379883
1800: 2.1116323471069336
1850: 2.182030200958252
1900: 1.9139916896820068
1950: 2.4332942962646484
2000: 1.6465215682983398
2050: 2.3844

In [11]:
source_sentence = train_dataset[55].query
print(source_sentence)

['it', "'s", 'more']


In [12]:
x = QUERY.numericalize([source_sentence]).to(device)
x = x.flatten()
print(x.shape)
result = bradley_model.greedy_inference_one_sample(x)

torch.Size([3])
69 tensor(9.2571, device='cuda:0', grad_fn=<SelectBackward>)
355 tensor(7.5266, device='cuda:0', grad_fn=<SelectBackward>)
360 tensor(9.6401, device='cuda:0', grad_fn=<SelectBackward>)
407 tensor(8.9998, device='cuda:0', grad_fn=<SelectBackward>)
442 tensor(7.1108, device='cuda:0', grad_fn=<SelectBackward>)
639 tensor(10.8222, device='cuda:0', grad_fn=<SelectBackward>)
650 tensor(8.9032, device='cuda:0', grad_fn=<SelectBackward>)
738 tensor(7.7009, device='cuda:0', grad_fn=<SelectBackward>)
762 tensor(10.8743, device='cuda:0', grad_fn=<SelectBackward>)
910 tensor(7.5250, device='cuda:0', grad_fn=<SelectBackward>)
939 tensor(8.7564, device='cuda:0', grad_fn=<SelectBackward>)
1190 tensor(7.6765, device='cuda:0', grad_fn=<SelectBackward>)
1222 tensor(7.2362, device='cuda:0', grad_fn=<SelectBackward>)
1228 tensor(7.0740, device='cuda:0', grad_fn=<SelectBackward>)
1321 tensor(7.9070, device='cuda:0', grad_fn=<SelectBackward>)
1449 tensor(8.8518, device='cuda:0', grad_fn=<Sel

In [15]:
RESPONSE.vocab.itos[4160]

'carpet'

In [14]:
def batch_index_to_strings(trg):
  # trg = [sent_len, batch_size]
  temp = trg.transpose(0,1)
  for i, row in enumerate(temp):
    print(row)

batch_index_to_strings(result)

tensor([   2, 4160, 4160, 4160, 4160, 4160, 4160, 4160, 4160, 4160, 4160],
       device='cuda:0')
